In [9]:
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/lanacaldarevic/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/lanacaldarevic/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [28]:
import pandas as pd

df = pd.read_csv('movies_training_data.csv')
df

,movieId,title,genres,overview,production_companies,production_countries,spoken_languages,tagline,tag,cast,directors
0,1,Toy Story,"Animation, Adventure, Family, Comedy","Led by Woody, Andy's toys live happily in his ...",Pixar,United States of America,English,Hang on for the comedy that goes to infinity a...,"owned, imdb top 250, pixar, pixar, time travel...","Tom Hanks, Tim Allen, Don Rickles, Jim Varney,...",John Lasseter
1,2,Jumanji,"Adventure, Fantasy, Family",When siblings Judy and Peter discover an encha...,"TriStar Pictures, Interscope Communications, T...",United States of America,"English, French",Roll the dice and unleash the excitement!,"robin williams, time travel, fantasy, based on...","Robin Williams, Kirsten Dunst, Bradley Pierce,...",Joe Johnston
2,3,Grumpier Old Men,"Romance, Comedy",A family wedding reignites the ancient feud be...,"Lancaster Gate, Warner Bros. Pictures",United States of America,English,Still Yelling. Still Fighting. Still Ready for...,"funny, best friend, duringcreditsstinger, fish...","Walter Matthau, Jack Lemmon, Ann-Margret, Soph...",Howard Deutch
3,4,Waiting to Exhale,"Comedy, Drama, Romance","Cheated on, mistreated and stepped on, the wom...",20th Century Fox,United States of America,English,Friends are the people who let you be yourself...,"based on novel or book, chick flick, divorce, ...","Whitney Houston, Angela Bassett, Loretta Devin...",Forest Whitaker
4,5,Father of the Bride Part II,"Comedy, Family",Just when George Banks has recovered from his ...,"Touchstone Pictures, Sandollar Productions",United States of America,English,Just when his world is back to normal... he's ...,"aging, baby, confidence, contraception, daught...","Steve Martin, Diane Keaton, Martin Short, Kimb...",Charles Shyer
...,...,...,...,...,...,...,...,...,...,...,...
17482,95581,The Flying Fleet,"Adventure, Drama, Romance","Six friends, all hoping to become aviators, ar...",Metro-Goldwyn-Mayer,United States of America,NaN,NaN,"aircraft carrier, aviator, expulsion, midshipm...","Ramon Novarro, Ralph Graves, Anita Page, Alfre...",George W. Hill
17483,95583,Savages,"Crime, Drama, Thriller",Pot growers Ben and Chon face off against the ...,"Relativity Media, Ixtlan, Onda Entertainment, ...",United States of America,English,Young. Beautiful. Deadly.,"american abroad, dea, dea agent, enforcer, exp...","Taylor Kitsch, Blake Lively, Aaron Taylor-John...",Oliver Stone
17484,95591,Rat King,Thriller,18 year old Juri is seriously addicted to onli...,"Making Movies, Allfilm","Finland, Estonia",Finnish,Would you put your life on the line?,visually stunning,"Max Ovaska, Outi Mäenpää, Janne Virtanen, Juli...",Petri Kotwica
17485,95595,Bela Kiss: Prologue,"Horror, Mystery, Thriller","A true story, Bela Kiss was one of the the mos...",Mirror Maze,Germany,"English, German",Jede Legende fordert neues Blut,"muddled, suspense","Kristina Klebe, Rudolf Martin, Fabian Stumm, B...",Lucien Förstner


In [29]:
# fill missing data with ''
df.fillna('', inplace=True)
df

,movieId,title,genres,overview,production_companies,production_countries,spoken_languages,tagline,tag,cast,directors
0,1,Toy Story,"Animation, Adventure, Family, Comedy","Led by Woody, Andy's toys live happily in his ...",Pixar,United States of America,English,Hang on for the comedy that goes to infinity a...,"owned, imdb top 250, pixar, pixar, time travel...","Tom Hanks, Tim Allen, Don Rickles, Jim Varney,...",John Lasseter
1,2,Jumanji,"Adventure, Fantasy, Family",When siblings Judy and Peter discover an encha...,"TriStar Pictures, Interscope Communications, T...",United States of America,"English, French",Roll the dice and unleash the excitement!,"robin williams, time travel, fantasy, based on...","Robin Williams, Kirsten Dunst, Bradley Pierce,...",Joe Johnston
2,3,Grumpier Old Men,"Romance, Comedy",A family wedding reignites the ancient feud be...,"Lancaster Gate, Warner Bros. Pictures",United States of America,English,Still Yelling. Still Fighting. Still Ready for...,"funny, best friend, duringcreditsstinger, fish...","Walter Matthau, Jack Lemmon, Ann-Margret, Soph...",Howard Deutch
3,4,Waiting to Exhale,"Comedy, Drama, Romance","Cheated on, mistreated and stepped on, the wom...",20th Century Fox,United States of America,English,Friends are the people who let you be yourself...,"based on novel or book, chick flick, divorce, ...","Whitney Houston, Angela Bassett, Loretta Devin...",Forest Whitaker
4,5,Father of the Bride Part II,"Comedy, Family",Just when George Banks has recovered from his ...,"Touchstone Pictures, Sandollar Productions",United States of America,English,Just when his world is back to normal... he's ...,"aging, baby, confidence, contraception, daught...","Steve Martin, Diane Keaton, Martin Short, Kimb...",Charles Shyer
...,...,...,...,...,...,...,...,...,...,...,...
17482,95581,The Flying Fleet,"Adventure, Drama, Romance","Six friends, all hoping to become aviators, ar...",Metro-Goldwyn-Mayer,United States of America,,,"aircraft carrier, aviator, expulsion, midshipm...","Ramon Novarro, Ralph Graves, Anita Page, Alfre...",George W. Hill
17483,95583,Savages,"Crime, Drama, Thriller",Pot growers Ben and Chon face off against the ...,"Relativity Media, Ixtlan, Onda Entertainment, ...",United States of America,English,Young. Beautiful. Deadly.,"american abroad, dea, dea agent, enforcer, exp...","Taylor Kitsch, Blake Lively, Aaron Taylor-John...",Oliver Stone
17484,95591,Rat King,Thriller,18 year old Juri is seriously addicted to onli...,"Making Movies, Allfilm","Finland, Estonia",Finnish,Would you put your life on the line?,visually stunning,"Max Ovaska, Outi Mäenpää, Janne Virtanen, Juli...",Petri Kotwica
17485,95595,Bela Kiss: Prologue,"Horror, Mystery, Thriller","A true story, Bela Kiss was one of the the mos...",Mirror Maze,Germany,"English, German",Jede Legende fordert neues Blut,"muddled, suspense","Kristina Klebe, Rudolf Martin, Fabian Stumm, B...",Lucien Förstner


In [17]:
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

class TextCombinerTransformer(BaseEstimator, TransformerMixin):
    """Combine multiple text columns into a single column."""
    def __init__(self, columns_to_combine):
        self.columns_to_combine = columns_to_combine
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        combined_text = X[self.columns_to_combine].fillna('').apply(lambda x: ' '.join(x), axis=1)
        return combined_text

class TextCleanerTransformer(BaseEstimator, TransformerMixin):
    """Clean text by removing punctuation, stopwords, and applying lemmatization."""
    def __init__(self):
        self.stop_words = set(stopwords.words('english'))
        self.lemmatizer = WordNetLemmatizer()
        self.stemmer = PorterStemmer()
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        def clean_text(text):
            text = text.lower()
            text = re.sub(r'[^\w\s]', '', text)
            tokens = word_tokenize(text)
            tokens = [word for word in tokens if word not in self.stop_words]
            tokens = [self.lemmatizer.lemmatize(word) for word in tokens]
            tokens = [self.stemmer.stem(word) for word in tokens]
            return ' '.join(tokens)
        return X.apply(clean_text)

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/lanacaldarevic/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/lanacaldarevic/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/lanacaldarevic/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [33]:
# pipeline
columns_to_combine=['title', 'overview', 'tagline', 'genres', 'production_companies', 'production_countries', 'spoken_languages', 'tagline', 'cast', 'directors']

text_pipeline = Pipeline(steps=[
    ('combine_text', TextCombinerTransformer(columns_to_combine=columns_to_combine)),
    ('clean_text', TextCleanerTransformer()),
    ('tfidf', TfidfVectorizer())
])

In [34]:
X = text_pipeline.fit_transform(df.drop(columns=['tag'], axis=1))

In [35]:
X

<17487x65070 sparse matrix of type '<class 'numpy.float64'>'
	with 926034 stored elements in Compressed Sparse Row format>

In [36]:
from sklearn.metrics import hamming_loss, accuracy_score, precision_score, recall_score, f1_score, jaccard_score, average_precision_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.linear_model import LogisticRegression

# Assuming you have your data in X (features) and y (multilabel targets)

# Example data preparation
# Let's assume you have a list of lists for multilabel targets
y_multilabel = [['class1', 'class2'], ['class2'], ['class1', 'class3'], ['class1', 'class2', 'class3'], ['class3']]

# Convert multilabel targets to binary format
mlb = MultiLabelBinarizer()
y_binary = mlb.fit_transform(y_multilabel)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_binary, test_size=0.2, random_state=42)

# Example model training (using Logistic Regression as an example)
model = LogisticRegression()
model.fit(X_train, y_train)

# Example predictions
y_pred = model.predict(X_test)

# Hamming Loss
hl = hamming_loss(y_test, y_pred)
print(f'Hamming Loss: {hl}')

# Subset Accuracy
subset_acc = accuracy_score(y_test, y_pred)
print(f'Subset Accuracy: {subset_acc}')

# Precision, Recall, F1-Score for each label (micro average)
precision_micro = precision_score(y_test, y_pred, average='micro')
recall_micro = recall_score(y_test, y_pred, average='micro')
f1_micro = f1_score(y_test, y_pred, average='micro')
print(f'Precision (Micro): {precision_micro}')
print(f'Recall (Micro): {recall_micro}')
print(f'F1-Score (Micro): {f1_micro}')

# Jaccard Similarity Score (micro average)
jaccard_micro = jaccard_score(y_test, y_pred, average='micro')
print(f'Jaccard Similarity Score (Micro): {jaccard_micro}')

# Average Precision Score (micro average)
avg_precision_micro = average_precision_score(y_test, y_pred, average='micro')
print(f'Average Precision Score (Micro): {avg_precision_micro}')

# Confusion Matrix
conf_matrix = confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1))
print(f'Confusion Matrix:\n{conf_matrix}')


ValueError: Found input variables with inconsistent numbers of samples: [17487, 5]